In [1]:
from pathlib import Path
import random 

from manipylator import HeadlessSimulatedRobot, SimulatedRobot
from manipylator.utils import render_robot_from_template

# ManiPyLator Basics
ManiPyLator wraps around two main robotics libraries, robotics-toolbox-python & Genesis and tries to provide an interface to both simulated and analytical robotics. Below is an example of loading an arbitrary URDF and a forward kinematics sanity check with both libraries:

In [2]:
with render_robot_from_template("robots/empiric") as robot_urdf:
    manny = HeadlessSimulatedRobot(robot_urdf)
print(manny.model)

[I 07/25/25 00:18:46.407 500] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout
Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified


[Genesis] [00:19:03] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [00:19:03] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [00:19:03] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [00:19:03] [INFO] Consider setting 'performance_mode=True' in production to maximise runtime speed, if significantly increasing compilation time is not a concern.
[Genesis] [00:19:04] [INFO] Running on [NVIDIA GeForce MX250] with backend gs.cuda. Device memory: 1.95 GB.
[Genesis] [00:19:04] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [00:19:04] [WARNING] Scene.show_FPS is deprecated. Please use Scene.profiling_options.show_FPS
[Genesis] [00:19:04] [INFO] Scene <952a35f> created.
[Genesis] [00:19:04] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <d390595>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.


Authorization required, but no authorization protocol specified


[Genesis] [00:19:04] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <a870560>, morph: <gs.morphs.URDF(file='/tmp/tmpnhx8dyw0.urdf')>, material: <gs.materials.Rigid>.


[Genesis] [00:19:10] [INFO] Falling back to legacy URDF parser. Default values of physics properties may be off.
[Genesis] [00:19:10] [INFO] Applying offset to base link's pose with user provided value in morph.
[Genesis] [00:19:21] [INFO] Building scene <952a35f>...
[Genesis] [00:19:50] [INFO] Compiling simulation kernels...
[Genesis] [00:20:16] [INFO] Building visualizer...
[Genesis] [00:20:17] [INFO] Software rendering context detected. Shadows and plane reflection not supported. Beware rendering will be extremely slow.
ERobot: measured, 6 joints (RRRRRR), dynamics, geometry, collision
┌──────┬──────────────────┬───────┬──────────────────┬─────────────────────────────────────────────────────────────────────────────┐
│ link │       link       │ joint │      parent      │                             ETS: parent to link                             │
├──────┼──────────────────┼───────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────┤
│    

For an arbitrary poes, we'll calculate the forward kinematics using both RTB and Genesis and compare:

In [3]:
arbitrary_pose = [random.uniform(-1, 1) for _ in range(6)]
print(arbitrary_pose)

[-0.4720275388975417, -0.18300879977360895, 0.6536733910306669, 0.3516591658552317, -0.7404998970643253, 0.7387880912391709]


In [4]:
forward_kinematics_rtb = manny.model.fkine(arbitrary_pose)
forward_kinematics_rtb

  -0.8065    0.4478    0.386    -0.2354    
   0.4828    0.1219    0.8672    0.2015    
   0.3413    0.8858   -0.3145    0.7165    
   0         0         0         1         


In [5]:
translation_rtb = forward_kinematics_rtb.t
translation_rtb

array([-0.23542257,  0.20153512,  0.71652318])

In [6]:
simulated_robot = manny.visualizer.robot
ee = simulated_robot.get_link('end_effector')
scene = manny.visualizer.scene

simulated_robot.set_dofs_position(arbitrary_pose)
scene.step()

translation_gs = ee.get_pos().cpu()
translation_gs

tensor([-0.2354,  0.2015,  0.7165])

In [7]:
from numpy import allclose
allclose(translation_rtb, translation_gs, atol=1e-5)

True

Both approaches should return True above, which means they calculate comparable solutios down to the 10 micron range

# Where To Now?
The above demonstrates the two available interfaces in ManiPylator. Based on your interests you might continue to explore our [visual robotics demonstration](https://nbviewer.org/github/leogolds/ManiPylator/blob/main/20-simulation.ipynb) or maybe try to derive a [symbolic result](https://nbviewer.org/github/leogolds/ManiPylator/blob/main/11-differntial-kinematics.ipynb). For whatever path you're interested in, check out our Robotics resources page.

For the curious, we recommend you jump straight into [controling Manny](https://nbviewer.org/github/leogolds/ManiPylator/blob/main/30-controlling-manny.ipynb) where you can try out a simulated version of Manny the ManiPylator. 

For experienced robotocists, check out our [FAQ](docs/FAQ.MD) & [Architecture](docs/ARCHITECTURE.md). For inspiration, see [parallel simulation](https://genesis-world.readthedocs.io/en/latest/user_guide/getting_started/parallel_simulation.html), [drone hovering](https://genesis-world.readthedocs.io/en/latest/user_guide/getting_started/hover_env.html) & [soft robotics](https://genesis-world.readthedocs.io/en/latest/user_guide/getting_started/soft_robots.html) examples from the [Genesis docs](https://genesis-world.readthedocs.io/en/latest/user_guide/getting_started/hello_genesis.html).

For teachers/students, see our notebooks starting with our symbolic manipulation notebooks, starting with 1X. These introduce [SymPy](https://docs.sympy.org/) and demonstrate some basic examples, demonstarte how to manipulate [SE2 & SE3](https://bdaiinstitute.github.io/spatialmath-python/intro.html#spatial-math-classes) objects and deriving some basic robotics results.